In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

import time

from distributed_utilities import *

Setting environment for Gordon


No vtk


In [15]:
stack = 'MD642'

In [25]:
border_dissim_percentile = 30
min_size = 100
filenames = [fn for fn in metadata_cache['sections_to_filenames'][stack].values() if not is_invalid(fn)]

In [26]:
exclude_nodes = [33]

In [ ]:
t = time.time()
print 'Generating thumbnail mask...',

input_dir = os.path.join(RAW_DATA_DIR, stack)
output_dir = create_if_not_exists(os.path.join(DATA_DIR, stack, stack + '_submasks'))

script_name = 'generate_thumbnail_masks_v4.py'
# !! For some reason (perhaps too much simultaneous write to disk), the distributed computation cannot finish, usually stuck with only a few sections left.

run_distributed4(command='%(script_path)s %(stack)s %(input_dir)s \'%%(filenames)s\' %(output_dir)s --border_dissim_percentile %(border_dissim_percentile)d --min_size %(min_size)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'preprocess', script_name),
                'stack': stack,
                'input_dir': input_dir,
                'output_dir': output_dir,
                'border_dissim_percentile': border_dissim_percentile,
                'min_size': min_size},
                kwargs_list=dict(filenames=filenames),
                exclude_nodes=exclude_nodes,
                argument_type='list2')

print 'done in', time.time() - t, 'seconds' # 500s

In [ ]:
# ... Review in local machine using GUI...

In [27]:
xmin, xmax, ymin, ymax = metadata_cache['cropbox'][stack]
anchor_fn = metadata_cache['anchor_fn'][stack]

In [28]:
t = time.time()
print 'warping thumbnail mask...',

elastix_output_dir = os.path.join(DATA_DIR, stack, stack+'_elastix_output')
transforms_filename = os.path.join(elastix_output_dir, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                                        dict(stack=stack, anchor_fn=anchor_fn))
transforms_to_anchor = pickle.load(open(transforms_filename, 'r'))

execute_command('rm -rf %(aligned_dir)s' % dict(aligned_dir=os.path.join(DATA_DIR, stack, stack + '_masks_alignedTo_' + anchor_fn)))

run_distributed4('%(script_path)s %(stack)s %(input_dir)s %(aligned_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 black' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'preprocess', 'warp_crop_IM_v2.py'),
                'stack': stack,
                'input_dir': os.path.join(DATA_DIR, stack, stack + '_masks'),
                'aligned_dir': os.path.join(DATA_DIR, stack, stack + '_masks_alignedTo_' + anchor_fn)},
                kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                            'filename': fn + '_mask.png',
                            'output_fn': fn + '_mask_alignedTo_' + anchor_fn + '.png'}
                            for fn in filenames],
                exclude_nodes=exclude_nodes,
                argument_type='single')

print 'done in', time.time() - t, 'seconds' # 350s

warping thumbnail mask... rm -rf /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD642/MD642_masks_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157
['gcn-20-33.sdsc.edu'] are excluded


Child returned 0


Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']
done in 366.522112846 seconds


In [29]:
t = time.time()
sys.stderr.write('cropping thumbnail mask...')

aligned_cropped_dir = '%(stack_data_dir)s/%(stack)s_masks_alignedTo_%(anchor_fn)s_cropped' % \
                        {'stack': stack,
                        'stack_data_dir': os.path.join(DATA_DIR, stack),
                        'anchor_fn': anchor_fn}

execute_command('rm -rf %(aligned_cropped_dir)s' % dict(aligned_cropped_dir=aligned_cropped_dir))

os.system(('mkdir %(aligned_cropped_dir)s ; '
            'mogrify -set filename:name %%t -crop %(w)dx%(h)d+%(x)d+%(y)d -write "%(aligned_cropped_dir)s/%%[filename:name]_cropped.png" %(stack_data_dir)s/%(stack)s_masks_alignedTo_%(anchor_fn)s/*.png') % \
    {'stack': stack,
    'stack_data_dir': os.path.join(DATA_DIR, stack),
    'aligned_cropped_dir': aligned_cropped_dir,
    'w':xmax+1-xmin, 'h':ymax+1-ymin, 'x':xmin, 'y':ymin,
    'anchor_fn': anchor_fn})

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70s

rm -rf /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD642/MD642_masks_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped


cropping thumbnail mask...Child returned 0
done in 61.756742 seconds
